In [ ]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SparkSession
import frameless.functions.aggregate._
import frameless.TypedDataset
import org.apache.spark.sql.SparkSession
import org.bdgenomics.adam.rdd.ADAMContext._
import comp.bio.aging.playground.extensions._
import org.apache.spark.sql._
import scala.util.Try
import org.apache.spark.storage.StorageLevel

val spark = SparkSession
  .builder()
  .appName("Homology search")
  .getOrCreate()
// For implicit conversions like converting RDDs to DataFrames
import spark.implicits._
spark.sparkContext.setLogLevel("WARN")

import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SparkSession
import frameless.functions.aggregate._
import frameless.TypedDataset
import org.apache.spark.sql.SparkSession
import org.bdgenomics.adam.rdd.ADAMContext._
import comp.bio.aging.playground.extensions._
import org.apache.spark.sql._
import scala.util.Try
import org.apache.spark.storage.StorageLevel
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1b7df6cb
import spark.implicits._


In [ ]:
val base = "hdfs://namenode/pipelines"
val diamond = base + "/diamond"
val blastp = diamond + "/blastp"
val graywhale = base + "/GRAY_WHALE"
val trinity = graywhale + "/Trin_Mitya.Trinity.fasta"
val proteins = graywhale + "/Trin_Mitya.Trinity.fasta.transdecoder.pep"
val gff = graywhale + "/Trin_Mitya.Trinity.fasta.transdecoder.gff3"

def readTSV(path: String) = spark.read.option("sep", "\t").csv(path)

base: String = hdfs://namenode/pipelines
diamond: String = hdfs://namenode/pipelines/diamond
blastp: String = hdfs://namenode/pipelines/diamond/blastp
graywhale: String = hdfs://namenode/pipelines/GRAY_WHALE
trinity: String = hdfs://namenode/pipelines/GRAY_WHALE/Trin_Mitya.Trinity.fasta
proteins: String = hdfs://namenode/pipelines/GRAY_WHALE/Trin_Mitya.Trinity.fasta.transdecoder.pep
gff: String = hdfs://namenode/pipelines/GRAY_WHALE/Trin_Mitya.Trinity.fasta.transdecoder.gff3
readTSV: (path: String)org.apache.spark.sql.DataFrame


In [ ]:
val proteinsAdam = graywhale + "/proteins.adam"
val transcriptsAdam = graywhale + "/transcripts.adam"

def convert() = {
  val transcripts = sparkContext.loadFasta(trinity)
  val query = sparkContext.loadFasta(proteins)
  query.saveAsParquet(proteinsAdam)
}


proteinsAdam: String = hdfs://namenode/pipelines/GRAY_WHALE/proteins.adam
transcriptsAdam: String = hdfs://namenode/pipelines/GRAY_WHALE/transcripts.adam
convert: ()Unit


In [ ]:
val transcripts = sparkContext.loadParquetContigFragments(transcriptsAdam)
val query = sparkContext.loadParquetContigFragments(proteinsAdam)


transcripts: org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD = ParquetUnboundNucleotideContigFragmentRDD with 114233 reference sequences
query: org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD = ParquetUnboundNucleotideContigFragmentRDD with 32429 reference sequences


Now, let's write classes to extract headers from ORF predictions

In [ ]:
import scala.util.Try
trait ProteinSearch{
  def id: String
  def e: String
}

case class BlastResult(id: String, score: Double, e: String) extends ProteinSearch
case class PfamResult(domain: String, id: String, e: String) extends ProteinSearch

case class ProteinPrediction(transcript: String,
                             id: String,
                             sequence: String,
                             orf_type: String,
                             score: Double,
                             start: Long,
                             end: Long,
                             len: Int,
                             strand: Char,
                             diamondHits: List[BlastResult],
                             pfamHits: List[PfamResult]
                            )

def extractPrediction(description: String, sequence: String): ProteinPrediction = {
    val gene::transcript::rest::Nil = description.split("::").toList
    val id::orf::tp::len_string::params::tr::Nil = rest.split(' ').filter(_!="").toList
    val orf_type: String = tp.substring(tp.indexOf(':') + 1)
    val _::etc::Nil = tr.split(':').toList
    val str::score_string::other = params.split(',').toList
    val strand: Char = if(str.contains("-")) '-' else '+'
    val len = Integer.parseInt(
      len_string.substring(len_string.indexOf(':') + 1)
    )
    val (diamondHits, pfamHits) = other.foldLeft((List.empty[BlastResult], List.empty[PfamResult])){
      case ((b, p), el) =>
        val id::value::e::Nil = el.split('|').toList
        Try(value.toDouble).map(
          v=>
            (BlastResult(id, v, e)::b, p)
        ).getOrElse(
          (b, PfamResult(id, value, e)::p)
        )
    }
    val span = etc.substring(0, etc.indexOf('('))
    val start::end::Nil = span.split('-').map(_.toLong).toList
    val score = score_string.substring(score_string.indexOf('=') + 1).toDouble
    ProteinPrediction(
      transcript, id, sequence, orf_type, score, start, end, len, strand, diamondHits, pfamHits
    )
  }


import scala.util.Try
defined trait ProteinSearch
defined class BlastResult
defined class PfamResult
defined class ProteinPrediction
extractPrediction: (description: String, sequence: String)ProteinPrediction


In [ ]:
val predictions = query.rdd
  .map(q=>extractPrediction(q.getDescription, q.getSequence))
  .filter(p=>p.diamondHits.nonEmpty || p.pfamHits.nonEmpty)
  .map(p=>(p.transcript, p))

predictions: org.apache.spark.rdd.RDD[(String, ProteinPrediction)] = MapPartitionsRDD[4] at map at <console>:110


In [ ]:
predictions.first

In [ ]:
val minky = "file:////pipelines/indexes/MINKY_WHALE/GCF_000493695.1_BalAcu1.0_rna.fna"
val t = sparkContext.loadFasta(minky)

minky: String = file:////pipelines/indexes/MINKY_WHALE/GCF_000493695.1_BalAcu1.0_rna.fna
t: org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD = RDDBoundNucleotideContigFragmentRDD with 37868 reference sequences


In [ ]:
t

<console>:69: error: not found: value t
       t
       ^


In [ ]:
println("Hello")

Hello


Where:
------
	qseqid means Query Seq - id
	qlen means Query sequence length
	sseqid means Subject Seq - id
	sallseqid means All subject Seq - id(s), separated by a ';'
	slen means Subject sequence length
	qstart means Start of alignment in query
	qend means End of alignment in query
	sstart means Start of alignment in subject
	send means End of alignment in subject
	qseq means Aligned part of query sequence
	sseq means Aligned part of subject sequence
	evalue means Expect value
	bitscore means Bit score
	score means Raw score
	length means Alignment length
	pident means Percentage of identical matches
	nident means Number of identical matches
	mismatch means Number of mismatches
	positive means Number of positive - scoring matches
	gapopen means Number of gap openings
	gaps means Total number of gaps
	ppos means Percentage of positive - scoring matches
	qframe means Query frame
	btop means Blast traceback operations(BTOP)
	staxids means unique Subject Taxonomy ID(s), separated by a ';' (in numerical order)
	stitle means Subject Title
	salltitles means All Subject Title(s), separated by a '<>'
	qcovhsp means Query Coverage Per HSP
	qtitle means Query title
